In [2]:
import kafka
from kafka import KafkaProducer
import pandas as pd
from json import dumps
import json
import time
import requests
import random
import creds


C:\Users\Ivan Shamoon\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [12]:
def extract_data():
    id_type = {'id_num': random.randint(1,999), 'type':random.choice(['tv','movie'])}
    response = requests.get("https://api.themoviedb.org/3/{medium}/{movie_num}?api_key={key}".format(medium = id_type["type"],movie_num = id_type["id_num"], key = creds.api_key))
    json_object = json.loads(response.content)
    json_object['type'] = id_type["type"]
    response_code = response.status_code
    return json_object, response_code

In [13]:
def clean_dic(dictionary):
    
    
    tv_keys = ["name","vote_average","first_air_date","type"]
    movie_keys = ["original_title","vote_average","release_date","type"]
    
    if dictionary['vote_average'] != 0.0:
        for key in list(dictionary.keys()):

            if dictionary["type"] == "tv":
                if key not in tv_keys:
                    dictionary.pop(key)

            else:
                if key not in movie_keys:
                    dictionary.pop(key)

        if dictionary["type"] == "tv":
            dictionary["original_title"] = dictionary.pop("name")
            dictionary["release_date"] = dictionary.pop("first_air_date")
        
        return dictionary

    

In [14]:
def send_data(producer, data, topic):
    time.sleep(1)
    if data != None:
        producer.send(topic,value = data)
        

In [15]:
def main_function(producer, topic):
    
    json_object, response_code = extract_data()
    
    if response_code == 200:
        
        new_dict = clean_dic(json_object)
        
        send_data(producer, new_dict, topic)
        


In [20]:
producer = KafkaProducer(bootstrap_servers = [creds.ip_host],
                        value_serializer = lambda x: json.dumps(x).encode('utf-8'))



if __name__ == "__main__":
    while True:
        data = main_function(producer, creds.topic_name)


KeyboardInterrupt: 